# households demo: modelling and household lifecycles #

Model construction has a long history in archaeology, but most models are still applied without explicit definition. In the realm of household archaeology, namely the study of the physical remains of houses and the attempts to reconstruct the social lives of their inhabitants, assumptions about the timescale of household growth and house renovation regularly rely on the temporality of events in the lifecycle. These assumptions are sometimes rooted in ethnographic observations of Mediterranean or world villages, while other times they are based on the author's intuition or personal experiences. Regardless of the justification, such models can be more profitably explored by explicit definition and construction. 

An explicit computational model of household lifecycles and house renovation can be constructed using the households Python package. I am writing this package to provide a consistent simulation toolbox for modelling ancient households as well as better understand the intersection between demographic processes and sociocultural transformations of daily life. The package is designed with comparability and easy parametrization in mind, making it possible to 